# House Price Detection Project


In [ ]:
import pandas as pd

In [ ]:
housing = pd.read_csv("train.csv")
housing.shape

In [ ]:
housing.head()                 
# '''shows the first 5 rows in the data'''

In [ ]:
housing.describe()  
# '''shows description of the data and relevant statistics'''


In [ ]:
housing['LONGITUDE'].value_counts()
# housing.value_counts()
# '''values_count gives values for for the wjole data as well as for 1 particular indices'''

In [ ]:
housing['LATITUDE'].describe()
# '''It describes 1 particular indices with all the statistics'''

In [ ]:
%matplotlib inline                                  
# ''' This command asks the notebook to show the graphs here itself'''
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))          #this command shows the histograph of the data 
plt.show()

In [ ]:
plt.show
import numpy as np

## Train- Test data splitting

In [ ]:
def split_train_test(data, test_ratio):                         #function is defined to split the train and test data
    shuffel = np.random.permutation(len(data))          # shuffels the data by random module from np
    print (shuffel)
    test_set_size = int(len(data) * test_ratio)                 # here determining the size of the test data(lenghth of data * test ratio=20%=0.2)  
    test_indices = shuffel[:test_set_size]                      # it says to shuffel the data from 0: size of test data
    train_indices = shuffel[test_set_size:]                            # it says to shuffel the data from the size of tewst data : 0
    return data.iloc[train_indices], data.iloc[test_indices]        
    

here the problem is due to random module if we run the program again and again the model will train ont the whole train data and no unseen data will be left for the testing due to this the model will overfit itself on this particular data so wee need to fix this keeping the data should be distributed equally to assure all possiblities are there in the train data

In [ ]:
train_set, test_set = split_train_test(housing, 0.0001)            
# '''giving atributes to the function and determiing the function = test_set, train_set'''
print (f"Rows in train set: {len(train_set)}")
print (f"rows in test set: {len(test_set)}")                                                                      

## By sklearn test and train set

the whole splitting can be done by sklearn easily as seen under the above thing was for the explanation how it works under the hood
also it does the stratified sampling of the data so that some part of the data is left unseen for test set

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, train_size=0.2, random_state=42)
print(f"Rows in train set: {len(train_set)} \nRows in test set: {len(test_set)}")

In [ ]:
# from sklearn.model_selection import train_test_split
# train_set, test_set = train_test_split(housing, train_size=0.2, random_state=42)
# print(f"Rows in train set: {len(train_set)} \nRows in test set: {len(test_set)}")

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit                  # it performs the stratified sampling 
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)      #test size is splitted into 20% and random_state is always 42
for train_index, test_index in split.split(housing, housing["RESALE"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_train_set.describe()
strat_train_set["RESALE"].value_counts
strat_train_set.info()

In [ ]:
housing = strat_train_set.copy()

## Looking for Correlations


In [ ]:
corr_matrix = housing.corr()
corr_matrix['TARGET(PRICE_IN_LACS)'].sort_values(ascending=False)
# '''This shows the correlation of price with other features and accr. to this price is strongly correlated with sq.ft'''

In [ ]:
from pandas.plotting import scatter_matrix
atributes = ['TARGET(PRICE_IN_LACS)', 'SQUARE_FT', 'BHK_NO.', 'RESALE', 'POSTED_BY']
scatter_matrix(housing[atributes], figsize=(12,10))

In [ ]:
housing.plot(kind="scatter" , x="BHK_NO.", y="TARGET(PRICE_IN_LACS)", alpha=1)

## Trying Attribute Combinations

In [ ]:
housing.plot(kind="scatter" , x="SQUARE_FT", y="TARGET(PRICE_IN_LACS)", alpha=1)

In [ ]:
housing["PRICESQFT"] = housing['TARGET(PRICE_IN_LACS)']/housing['SQUARE_FT']

In [ ]:
housing['PRICESQFT']

In [ ]:
housing.head

In [ ]:
housing["MOVE_RESALE"] = housing['READY_TO_MOVE']/housing['RESALE']
housing["MOVE_RESALE"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix['TARGET(PRICE_IN_LACS)'].sort_values(ascending=False)

In [ ]:
housing["SQUARE_RESALE"] = housing['SQUARE_FT']/housing['RESALE']
corr_matrix = housing.corr()
corr_matrix['TARGET(PRICE_IN_LACS)'].sort_values(ascending=False)

In [ ]:
housing.plot(kind="scatter" , x="SQUARE_FT", y="RESALE", alpha=1)

In [ ]:
housing.plot(kind="scatter" , x="SQUARE_RESALE", y="TARGET(PRICE_IN_LACS)", alpha=1)

In [ ]:
housing = strat_train_set.drop("TARGET(PRICE_IN_LACS)", axis=1)
housing_labels = strat_train_set["TARGET(PRICE_IN_LACS)"].copy()

##  Missing Attributes

In [ ]:
# To take care of the missing attributes we have 3 options
# 1, Get rid of the missing data poins
# 2, get rid of the wjole attribute
# 3, set the value to some value (0, mean, median) 

In [ ]:
# a = housing.dropna(subset=['RESALE']) # option 1
# a.shape
'''Till we doesnt writ (,housing) we cannot update the data of housing we jus get a copy ofit in (a)'''

In [ ]:
# housing.dropna('RESALE', axis=1) # option 2 
'''The whole RESALE column will not be there here also the orignal housing data frame will be unchanged same as above'''

In [ ]:
# median = housing['RESALE'].median()   # option 3
# housing['RESALE'].fillna(median)
'''this changes the missing values and fill the median over there if there are missing values in the train set we also have to edit the test set'''

In [ ]:
from sklearn.impute import SimpleImputer                            # for filling the missing values as median values 
imputer = SimpleImputer(strategy='median')
# imputer = SimpleImputer(strategy = "median")
imputer.fit(housing)
# imputer.statistics_.shape
# imputer.statistics
# 

In [ ]:
imputer.statistics_
X = imputer.transform(housing)
housing_tr = pd.DataFrame(X, columns=housing.columns)                       #using imputer
housing_tr.describe()



## Scikit-Learn Design

In [ ]:
'''Priarily three types of objects
1. Estimators - It estimates some parameters (eg. imputer) it also has a fit method
    it fits the data set and calculates the intrnal parameters.

2. Transformers - it takes input and returns outout based in the learnng on fit method fit(). 
    it also has a convenience func. called fit_transform() which fits and then transforms.

3. Predictors - LinearRegression model is and example of predictor. fit() and predict are two common func. 
    It also gives score func which will evaluate the predictions.'''

## Feature scalling

In [ ]:
'''Two types of feature scalling methods:
1. Min-Max scalling (normalization)
    (value - min) / (max - min)
    Sklearn provies class calle MinMaxScaler for this

2. Standardization
    (value - min) / std 
    Sklearn provides a class called StandaredScaler for this'''

## Creating Pipeline 

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),                          # Creating a pipeline 
    # .... add as many you want in your pipeline
    ('std_scaler', StandardScaler()),
])

In [ ]:
housing_num_tr = my_pipeline.fit_transform(housing_tr)
housing_num_tr.shape                                                        # Transforming the model along with fit

## Selecting a desiered model 

In [ ]:
'''# from sklearn.linear_model import LinearRegression
# from sklearn.tree import DecisionTreeRegressor              # As linear regresson didnt worked we used decision tree
from sklearn.ensemble import RandomForestRegressor              # As RandomForest Regressor
# model = DecisionTreeRegressor()                               # As DecisionTreeRegressor
model = RandomForestRegressor()
# model = LinearRegression()
model.fit(housing_num_tr, housing_labels)'''

In [ ]:
'''#some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
prepared_data = my_pipeline.transform(some_data)
model.predict(prepared_data)''''


In [ ]:
# list (some_labels)

## Evaluating the Model

In [ ]:
'''from sklearn.metrics import mean_squared_error
housing_predictions = model.predict(housing_num_tr)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
# mse = mean_squared_error(housing_labels, housing_predictions)   # For decision tree 
lin_rmse = np.sqrt(lin_mse)
# rmse = np.sqrt(mse)   # for decision tree '''


In [ ]:
# lin_rmse
# rmse

## using better evaluation technique - Cross Validation

In [ ]:
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(model, housing_num_tr, housing_labels, scoring = "neg_mean_squared_error", cv= 10)
# rmse_scores = np.sqrt(-scores)

In [ ]:
# rmse_scores

In [ ]:
# def print_scores(scores):
#     print("Scores:", scores)
#     print("Mean", scores.mean())
#     print("Standard deviation:", scores.std())


In [ ]:
# print_scores(rmse_scores)

In [ ]:
'''Dumping the model '''

from joblib import dump, load
dump(model, 'Dragon.joblib')
